# Basic objects

> Basic object types and methods utilized in `pybx`

In [32]:
# | default_exp basics

In [33]:
# | export
import warnings
import inspect
from typing import Union, Dict
from numpy.typing import ArrayLike

import numpy as np
from fastcore.dispatch import explode_types
from fastcore.foundation import L, noop
from fastcore.basics import concat, store_attr, patch, GetAttr
from fastcore.xtras import is_listy

from pybx.ops import (
    mul,
    sub,
    intersection_box,
    make_single_iterable,
    voc_keys,
    label_keys,
    update_keys,
)
from pybx.excepts import *

COORD_TYPES = (np.int_, int, np.float_)
ITER_TYPE_JSON = Dict[str, str]
ITER_TYPES = (np.ndarray, list, L)
ITER_TYPES_TUPLE = (tuple,)
ITER_TYPES_EXTRA = (dict, ITER_TYPE_JSON)
ITER_TYPE_HASHED = (dict, ITER_TYPE_JSON)
ALL_ITER_TYPES = ITER_TYPES + ITER_TYPES_TUPLE + ITER_TYPES_EXTRA
ALL_TYPES = COORD_TYPES + ITER_TYPES

In [34]:
import json
from fastcore.test import test_eq, test_fail, test_warns, ExceptionExpected

# Check validity of boxes

In [35]:
# | export


def check_format_types(b: list, verbose=False):
    """
    Checks if the provided bounding box is in the correct format, typically of [int, int, int, int, str].

    Args:
        b (list): bounding box coordinates with or without label

    Returns:
        bool
    """
    result = all(map(lambda x: isinstance(x, (COORD_TYPES, str)), b))
    if verbose:
        verbose_result = "Passed" if result else "Failed"
        print(f"{verbose_result} `check_format_types`.")
    return result

In [36]:
list_bbox = [14, 51, 71, 92, "item"]
list_bbox_nolabel = [14, 51, 71, 92]

In [37]:
check_format_types(list_bbox), check_format_types(list_bbox_nolabel)

(True, True)

In [38]:
np.array(list_bbox_nolabel).dtype

dtype('int64')

In [39]:
check_format_types(np.array(list_bbox_nolabel), verbose=True)

Passed `check_format_types`.


True

In [40]:
# | export


def check_length_types(b: list, verbose=False):
    """
    Checks if the provided bounding box has an acceptable length and type,
    typically of [int, int, int, int, str] or [int, int, int, int]

    Also checks that all values are positive.

    Args:
        b (list): bounding box coordinates with or without label

    Returns:
        bool
    """
    b_label = True
    # check if len of the coordinates are 4
    b_len = len(b[:4]) == 4
    # check if first 4 items are int and positive
    b_coord_pos = all(
        map(lambda x: x >= 0 if isinstance(x, COORD_TYPES) else False, b[:4])
    )
    # if labels provided, check if string
    if len(b) > 4:
        b_label = all(map(lambda x: isinstance(x, str), b[4:]))

    result = all((b_len, b_coord_pos, b_label))
    if verbose:
        verbose_result = "Passed" if result else "Failed"
        print(
            f"{verbose_result} `check_length_types`. Results were label:{b_label}, len:{b_len}, coords:{b_coord_pos}"
        )
    return result

In [41]:
check_length_types(list_bbox), check_length_types(list_bbox_nolabel)

(True, True)

In [42]:
(
    check_length_types([14, 51, 92, "item1"]),
    check_length_types([14, 51, 92]),
    check_length_types([14, 51, -1, 92]),
)

(False, False, False)

In [43]:
check_length_types([14, 51, 92, "item1"], verbose=True)

Failed `check_length_types`. Results were label:True, len:True, coords:False


False

In [44]:
# | export


def check_max_voc(b: list, verbose=False):
    """
    Checks if the provided bounding box bottom right corner (x_max, y_max) is greater
    than the top left corner (x_min, y_min), which is true for voc format.

    Args:
        b (list): bounding box coordinates with or without label

    Returns:
        bool
    """
    assert len(b) >= 4, f"Not enough items in passed bounding box {b}."
    b_coord = b[:4]
    xs, ys = b_coord[::2], b_coord[1::2]
    assert all(
        map(lambda x: isinstance(x, COORD_TYPES), xs)
    ), f"Got `x_min`, `x_max` of wrong type {xs}"
    assert all(
        map(lambda x: isinstance(x, COORD_TYPES), ys)
    ), f"Got `y_min`, `y_max` of wrong type {ys}"

    check_xs = xs[1] > xs[0]
    check_ys = ys[1] > ys[0]
    result = check_xs and check_ys
    if verbose:
        verbose_result = "Passed" if result else "Failed"
        print(
            f"{verbose_result} `check_max_voc`. Results were check_xs:{check_xs}, check_ys:{check_ys}"
        )
    return result

In [45]:
list_bbox = [14, 51, 71, 92, "item"]
list_bbox_nolabel = [14, 51, 71, 92]

In [46]:
check_max_voc(list_bbox), check_max_voc(list_bbox_nolabel)

(True, True)

In [47]:
check_max_voc([14, 51, 10, 92]), check_max_voc([0, -1, 10, 92])

(False, True)

In [48]:
check_max_voc([14, 51, 10, 92], verbose=True)

Failed `check_max_voc`. Results were check_xs:False, check_ys:True


False

Method that does all of the above checks.

In [49]:
# | export


def perform_box_checks(b: list, verbose=False):
    """Calls all checks for bounding boxes: check_max_voc, check_length_types, check_format_types."""
    checks = (
        check_max_voc(b, verbose=verbose),
        check_length_types(b, verbose=verbose),
        check_format_types(b, verbose=verbose),
    )
    return all(checks)

In [50]:
perform_box_checks([14, 51, 10, 92], verbose=True)

Failed `check_max_voc`. Results were check_xs:False, check_ys:True
Passed `check_length_types`. Results were label:True, len:True, coords:True
Passed `check_format_types`.


False

# Parse bounding boxes

Bounding box coordinates of type `list`/`dict`/`json`/`array` can be converted 
to a `Bx` instance. Once wrapped as a `Bx` instance, some interesting properties can
be calculated from the coordinates. 

In [51]:
# | export


def parse_list(b: Union[list, ArrayLike], verbose=False, no_check=False):
    """
    Takes a list and splits into bounding box coordinates and label

    Args:
        b (list): Bounding box coordinates with or without label

    Returns:
        list: Bounding box coordinates
        str: Bounding box label
    """
    if not no_check:
        assert perform_box_checks(
            b, verbose=verbose
        ), f"Failed `perform_box_checks` for bounding box coordinates {b}"
    coords = list(b[:4])
    label = "unknown"
    if len(b) > 4:
        label = b[-1]
    return [coords], [label]

In [52]:
parse_list([10, 0, 11, 6])

([[10, 0, 11, 6]], ['unknown'])

In [53]:
list_bbox = [14, 51, 71, 92, "item"]
list_bbox_nolabel = [14, 51, 71, 92]

In [54]:
parse_list(list_bbox), parse_list(list_bbox_nolabel)

(([[14, 51, 71, 92]], ['item']), ([[14, 51, 71, 92]], ['unknown']))

Works the same way with arrays.

In [55]:
perform_box_checks(np.array(list_bbox_nolabel))

True

In [56]:
parse_list(np.array(list_bbox_nolabel))

([[14, 51, 71, 92]], ['unknown'])

If wrong or bad bounding boxes are passed, it will throw an Error.

In [57]:
try:
    parse_list([14, 51, 71, -1])
except AssertionError as A:
    print(A)

Failed `perform_box_checks` for bounding box coordinates [14, 51, 71, -1]


In [58]:
try:
    parse_list([14, 51, 71])
except AssertionError as A:
    print(A)

Not enough items in passed bounding box [14, 51, 71].


In [59]:
try:
    parse_list([14, 51, 71, "item"])
except AssertionError as A:
    print(A)

Got `y_min`, `y_max` of wrong type [51, 'item']


Same operations with `dict` and `json` strings.

In [60]:
dict_bbox = {"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}
dict_bbox_labelkey = {
    "x_min": 14,
    "y_min": 51,
    "x_max": 71,
    "y_max": 92,
    "object": "item",
}
dict_bbox_nolabel = {"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92}

In [67]:
voc_keys

['x_min', 'y_min', 'x_max', 'y_max', 'label']

In [78]:
# | export


def parse_dict(b: dict, **kwargs):
    """
    Takes a dict and splits into bounding box coordinates and label

    Args:
        b (dict): Bounding box coordinates with or without label

    Returns:
        list: Bounding box coordinates
        str: Bounding box label
    """
    b_ = []
    keys = update_keys(b)
    for k in keys:
        try:
            b_.append(b[k])
        except KeyError:
            warnings.warn(f"No {k} key in {b}")
            pass
    return parse_list(b_, **kwargs)

In [79]:
update_keys(dict_bbox_nolabel)

['x_min', 'y_min', 'x_max', 'y_max', 'label']

In [80]:
parse_dict(dict_bbox), parse_dict(dict_bbox_labelkey), parse_dict(dict_bbox_nolabel)

/var/folders/sl/tgj9ztc51hz_d2yt8kg97f840000gn/T/ipykernel_15739/1918640422.py:21: UserWarning: No label key in {'x_min': 14, 'y_min': 51, 'x_max': 71, 'y_max': 92}
  warnings.warn(f"No {k} key in {b}")


(([[14, 51, 71, 92]], ['item']),
 ([[14, 51, 71, 92]], ['item']),
 ([[14, 51, 71, 92]], ['unknown']))

Same operations with Json strings.

In [63]:
json_bbox = '{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}'
json_bbox_labelkey = (
    '{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "object": "item"}'
)
json_bbox_nolabel = '{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92}'

In [64]:
# | export


def parse_json(b: Union[str, Dict], **kwargs):
    """
    Takes a json string or dict and splits into bounding box coordinates and label

    Args:
        b (json str): Bounding box coordinates with or without label

    Returns:
        list: Bounding box coordinates
        str: Bounding box label
    """
    if isinstance(b, str):
        b = json.loads(b)
    elif isinstance(b, dict):
        return parse_dict(b, **kwargs)
    else:
        raise NotImplementedError(f"Unknown type passed to `parse_json` {b}")
    return parse_dict(b, **kwargs)

In [66]:
json_bbox

'{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}'

In [65]:
parse_json(json_bbox), parse_json(json_bbox_labelkey), parse_json(json_bbox_nolabel)

(([[14, 51, 71, 92]], ['item']),
 ([[14, 51, 71, 92]], ['item']),
 ([[14, 51, 71, 92]], ['unknown']))

It could also be a json object read from file.

In [ ]:
json_bbox_file = json.load(open("../data/annots.json"))[0]
json_bbox_file

{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'}

In [ ]:
parse_json(json_bbox_file)

([[130, 63, 225, 180]], ['clock'])

In [ ]:
# | export
class Bx:
    """Interface for all future Bx's"""

    def __init__(self, coords, label: list = None, verbose=False, no_check=False):
        # number of points to represent a box
        n_points = 4
        if no_check:
            coords, parsed_label = coords, label
        else:
            # checks for box validity and parses coordinates
            coords, parsed_label = parse_list(coords)
        # make coord a list of lists
        coords = [coords] if not is_listy(coords[0]) else coords
        # make label a list of single item
        label = label if label else parsed_label
        label = label if is_listy(label) else [label]
        # internal representation as a list
        _coords = coords[0]
        x_min, y_min, x_max, y_max = _coords
        store_attr(
            "x_min, y_min, x_max, y_max, _coords, coords, label, verbose, no_check, n_points"
        )

    def __str__(self):
        return f"Bx(coords={self.coords}, label={self.label})"

    def __repr__(self):
        return self.__str__()

    def __len__(self):
        return len(self.label)

    def get_coords(self):
        return self.coords

    @property
    def coords_as_numpy(self):
        return np.array(self.coords, dtype=int)

    def get_label(self):
        return self.label

    @property
    def bw(self):
        """Calculate width"""
        return self.x_max - self.x_min

    @property
    def bh(self):
        """Calculate height"""
        return self.y_max - self.y_min

    @property
    def cx(self):
        """Calculate center-x"""
        return (self.x_min + self.x_max) / 2.0

    @property
    def cy(self):
        """Calculate center-y"""
        return (self.y_min + self.y_max) / 2.0

    @property
    def area(self):
        """Calculates the absolute value of the area of the box."""
        return abs(self.bw * self.bh)

    @property
    def values(self):
        """Returns the coordinates and label as a single list."""
        return L([[*self._coords, *self.label]])

    @property
    def valid(self):
        """Checks for validity of the box and returns a boolean.
        From `v0.1.3`, validity implies that the box has non-zero area.
        """
        return all(
            [(self.area > 0), (self.x_max > self.x_min), (self.y_max > self.y_min)]
        )

    @property
    def xywh(self):
        """Converts the `pascal_voc` bounding box to `coco` format."""
        return [[self.x_min, self.y_min, self.bw, self.bh, *self.label]]

    def yolo(self, w=1, h=1, normalize=False):
        """Converts the `pascal_voc` bounding box to `yolo` centroids format.
        :param normalize: Whether to normalize the bounding box with image width and height.
        :param w: Width of image. Not to be confused with `BaseBx` attribute `w`.
        :param h: Height of image. Not to be confused with `BaseBx` attribute `h`.
        """
        if normalize:
            assert (w > 1) and (
                h > 1
            ), f"{inspect.stack()[0][3]} of {__name__}: Expected width and height of image with normalize={normalize}."
        _yolo = np.array([self.cx, self.cy, self.bw, self.bh]) / np.tile([w, h], 2)
        _yolo = _yolo.round(4).tolist()
        _yolo.append(*self.label)
        return [_yolo]

Initializing an empty `Bx` class. It does a whole lot of things!

Generate random coordinates for one anchor boxes.

In [ ]:
np.random.seed(42)
annots = [sorted([np.random.randint(100) for i in range(4)])]
annots

[[14, 51, 71, 92]]

If a single list is passed, `Bx` will make it a list of list.

In [ ]:
annots[0]

[14, 51, 71, 92]

In [ ]:
b = Bx(annots[0])
b

Bx(coords=[[14, 51, 71, 92]], label=['unknown'])

In [ ]:
b.n_points

4

In [ ]:
len(b)

1

In [ ]:
b.cx

42.5

If label is passed along with coords, this is used.

In [ ]:
b = Bx(annots[0], label="item")
b

Bx(coords=[[14, 51, 71, 92]], label=['item'])

Can be a list of single item name also.

In [ ]:
b = Bx(annots[0], label=["item"])
b

Bx(coords=[[14, 51, 71, 92]], label=['item'])

In [ ]:
b.yolo

<bound method Bx.yolo of Bx(coords=[[14, 51, 71, 92]], label=['item'])>

To get normalized coordinates wrt to the image dimensions.

In [ ]:
b.yolo(224, 224, normalize=True)

[[0.1897, 0.3192, 0.2545, 0.183, 'item']]

In [ ]:
b.values

(#1) [[14, 51, 71, 92, 'item']]

In [ ]:
b.coords_as_numpy

array([[14, 51, 71, 92]])

In [ ]:
b.xywh

[[14, 51, 57, 41, 'item']]

Performs all checks for the validity of the box.

In [ ]:
b.valid

True

`Bx` is inherited by all other types in `pybx`: `BaseBx`, `MultiBx`, `ListBx`, `JsonBx`, exposing the same properties.

`BaseBx` works with other types of coordinates too. 
It accepts the coordinates and label for one anchor box in a `list` or `ndarray` 
format.

In [ ]:
# | export
class BaseBx(Bx):
    """BaseBx is the most primitive form of representing a bounding box.
    Coordinates and label of a bounding box can be wrapped as a BaseBx using:
    `bbx(coords, label)`.

    :param coords: can be of type `list` or `array` representing a single box.
        - `list` can be formatted with `label`: `[x_min, y_min, x_max, y_max, label]`
            or without `label`: `[x_min, y_min, x_max, y_max]`
        - `array` should be a 1-dimensional array of shape `(4,)`

    :param label: a `list` or `str` that has the class name or label for the object
    in the corresponding box.
    """

    def __init__(self, coords, label: list = None, no_check=False):
        self.index = 0  # Fixes #2, calls itself everytime
        assert isinstance(
            coords, (list, L, np.ndarray)
        ), f"{__name__}: Expected type list or np.ndarray for coords, got {type(coords)}"
        super().__init__(coords, label, no_check=no_check)

    def __str__(self):
        return f"BaseBx(coords={self.coords}, label={self.label})"

Works with arrays and lists:

In [ ]:
annots[0]

[14, 51, 71, 92]

In [ ]:
BaseBx(annots[0])

BaseBx(coords=[[14, 51, 71, 92]], label=['unknown'])

In [ ]:
b = BaseBx(annots[0], "flower")
b

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

In [ ]:
b.coords

[[14, 51, 71, 92]]

In [ ]:
b.coords_as_numpy

array([[14, 51, 71, 92]])

Calling the `values` attribute returns the labels along with the coordinates.

In [ ]:
b.values

(#1) [[14, 51, 71, 92, 'flower']]

A short cut function that calles `BaseBx` with a list of coordinates.

In [ ]:
# | export


def bbx(coords=None, label=None, no_check=False):
    """Alias of the `BaseBx` class using lists."""
    return BaseBx(coords, label, no_check=no_check)

Remember that `BaseBx` can only have one box coordinate and label at a time.

In [ ]:
annots_list = [
    [10, 20, 100, 200, "apple"],
    [40, 50, 80, 90, "coke"],
]

In [ ]:
annots_list[0]

[10, 20, 100, 200, 'apple']

In [ ]:
bbx(annots_list[0])

BaseBx(coords=[[10, 20, 100, 200]], label=['apple'])

Boxes with errors will not be initialized.

In [ ]:
try:
    bbx([10, 20, 100, -1, "apple"])
except AssertionError as A:
    print(A)

Failed `perform_box_checks` for bounding box coordinates [10, 20, 100, -1, 'apple']


In [ ]:
bbx(annots_list[0][:4])  # if label is not passed

BaseBx(coords=[[10, 20, 100, 200]], label=['unknown'])

`BaseBx` also exposes a method to calculate the Intersection Over Union (IOU) using the `intersection_box` method

In [ ]:
intersection_box([10, 10, 100, 100], [10, 10, 150, 150])

array([ 10,  10, 100, 100])

In [ ]:
# | export


@patch
def iou(self: BaseBx, other):
    """Caclulates the Intersection Over Union (IOU) of the box
    w.r.t. another `BaseBx`. Returns the IOU only if the box is
    considered `valid`, ie non-zero area.
    """
    if not isinstance(other, Bx):
        other = bbx(other)
    if self.valid:
        try:
            int_box = bbx(intersection_box(self.coords, other.coords))
        except NoIntersection:
            return 0.0
        int_area = int_box.area
        union_area = other.area + self.area - int_area
        return round(int_area / union_area, 4)
    return 0.0

In [ ]:
b

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

In [ ]:
b2 = bbx(annots_list[1])
b2

BaseBx(coords=[[40, 50, 80, 90]], label=['coke'])

In [ ]:
b.iou(b2)

0.4432

In [ ]:
b.iou(b)

1.0

`BaseBx` is also pseudo-iterable (calling an iterator returns `self` itself and not the coordinates or labels).

In [ ]:
# | export
@patch
def __iter__(self: BaseBx):
    """Iterates through the boxes in `BaseBx` where self.valid is True."""
    return self


@patch
def __getitem__(self: BaseBx, idx):
    """Gets the item at index idx as a BaseBx."""
    if idx > 0:
        # Fixes #2
        raise IndexError(
            f"BaseBx has only a single coordinate at idx=0. Got idx={idx}."
        )
    return self


@patch
def __next__(self: BaseBx):
    """Iteration is allowed only for valid boxes"""
    try:
        b = self[self.index]
        if not b.valid:
            # 0 area boxes are not valid
            self.index += 1
            return self.__next__()
    except IndexError:
        self.index = 0  # reset index
        raise StopIteration
    self.index += 1
    return b

In [ ]:
b = BaseBx(annots[0], "flower")

In [ ]:
next(b)

BaseBx(coords=[[14, 51, 71, 92]], label=['flower'])

In [ ]:
b = BaseBx(annots_list[0])
for b_ in b:
    print(b_)

BaseBx(coords=[[10, 20, 100, 200]], label=['apple'])


Working with `json` strings.

In [ ]:
json_bbox

'{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}'

In [ ]:
# | export


def jbx(coords=None, label=None):
    """Alias of the `BaseBx` class using json strings."""
    coords, parsed_label = parse_json(coords)
    # make label a list of single item
    label = label if label else parsed_label
    label = label if is_listy(label) else [label]
    # no_check since parse_json already checks
    return BaseBx(coords, label, no_check=True)

In [ ]:
jbx(json_bbox)  # json string

BaseBx(coords=[[14, 51, 71, 92]], label=['item'])

In [ ]:
jbx(json_bbox_file)  # read from file as python object

BaseBx(coords=[[130, 63, 225, 180]], label=['clock'])

Working with `dict`s.

In [ ]:
dict_bbox

{'x_min': 14, 'y_min': 51, 'x_max': 71, 'y_max': 92, 'label': 'item'}

In [ ]:
# | export


def dbx(coords=None, label=None):
    """Alias of the `BaseBx` class using dict."""
    coords, parsed_label = parse_dict(coords)
    # make label a list of single item
    label = label if label else parsed_label
    label = label if is_listy(label) else [label]
    # no_check since parse_dict already checks
    return BaseBx(coords, label, no_check=True)

In [ ]:
dbx(dict_bbox)

BaseBx(coords=[[14, 51, 71, 92]], label=['item'])

To make it work with other types, need to infer the datatype. Basically need to call the `parse_X` function based on the datatype, where `X=datatype`.

In [ ]:
ALL_ITER_TYPES

(numpy.ndarray,
 list,
 fastcore.foundation.L,
 tuple,
 dict,
 typing.Dict[str, str])

In [ ]:
ITER_TYPE_JSON

typing.Dict[str, str]

In [ ]:
# | export


def parse_basebx(b: BaseBx, no_check=None):
    """Reads the attribute of a BaseBx"""
    if no_check is not None:
        # no_check is typically passed when creating a basebx
        warnings.warn(
            f"no_check={no_check} passed to parse_basebx: are you sure you want to do this?"
        )
    return b.coords, b.label

In [ ]:
parse_basebx(b)

([[10, 20, 100, 200]], ['apple'])

In [ ]:
parse_basebx(b, no_check=False)

/tmp/ipykernel_332941/3032591698.py:8: UserWarning: no_check=False passed to parse_basebx: are you sure you want to do this?
  warnings.warn(


([[10, 20, 100, 200]], ['apple'])

In [ ]:
# | export


def infer_box_dtype(b: ALL_ITER_TYPES, **kwargs):
    if isinstance(b, str):
        return "json"
    elif isinstance(b, ITER_TYPES):
        return "list"
    elif isinstance(b, dict):
        return "dict"
    elif isinstance(b, BaseBx):
        return "basebx"
    elif isinstance(b, np.ndarray):
        return "array"
    else:
        raise NotImplementedError(
            f"Unknown type {type(b)} passed to `infer_box_dtype` {b}"
        )

In [ ]:
(
    infer_box_dtype(json_bbox),
    infer_box_dtype(list_bbox),
    infer_box_dtype(json_bbox_file),
    infer_box_dtype(b),
)

('json', 'list', 'dict', 'basebx')

Parser method to call the correct method based on the type

In [ ]:
# | export


def coord_parser(b, label="unknown", no_check=False):
    """Takes the box and converts it to a BaseBx after inferring the type."""
    if not is_listy(label):
        label = [label]
    box_dtype = infer_box_dtype(b)
    parser = eval(f"parse_{box_dtype}")
    coords, parsed_label = parser(b, no_check=no_check)
    if parsed_label[0] == "unknown":
        parsed_label = label
    return bbx(coords=coords[0], label=parsed_label, no_check=no_check)

In [ ]:
(
    coord_parser(json_bbox),
    coord_parser(list_bbox),
    coord_parser(json_bbox_file),
)

(BaseBx(coords=[[14, 51, 71, 92]], label=['item']),
 BaseBx(coords=[[14, 51, 71, 92]], label=['item']),
 BaseBx(coords=[[130, 63, 225, 180]], label=['clock']))

With `no_check`, the validity of the boxes are not checked.

In [ ]:
coord_parser([0, 0, 0, 0], no_check=True)

BaseBx(coords=[[0, 0, 0, 0]], label=['unknown'])

In [ ]:
(
    coord_parser(list_bbox_nolabel),
    coord_parser(list_bbox_nolabel, ["item"]),
    coord_parser(list_bbox_nolabel, "item"),
)

(BaseBx(coords=[[14, 51, 71, 92]], label=['unknown']),
 BaseBx(coords=[[14, 51, 71, 92]], label=['item']),
 BaseBx(coords=[[14, 51, 71, 92]], label=['item']))

# Multiple bounding boxes

Working with multiple bounding boxes and annotaions is usually done with the help
of `MultiBx`. `MultiBx` allows for iteration.

In [ ]:
annots_list  # a good candidate for Multibox

[[10, 20, 100, 200, 'apple'], [40, 50, 80, 90, 'coke']]

It could be imagined as such a thing, maybe thats all we need. But maybe it can be made prettier.

In [ ]:
primitive_mbx = [bbx(a) for a in annots_list]
primitive_mbx

[BaseBx(coords=[[10, 20, 100, 200]], label=['apple']),
 BaseBx(coords=[[40, 50, 80, 90]], label=['coke'])]

In [ ]:
primitive_mbx[0].iou(primitive_mbx[1])

0.0988

In [ ]:
is_listy(annots), is_listy(annots[0])

(True, True)

In [ ]:
# | export
class MultiBx:
    """`MultiBx` represents a collection of bounding boxes as lists.
    Objects of type `MultiBx` can be indexed into, which returns a
    `BaseBx` exposing a suite of box-bound operations.
    Multiple coordinates and labels of bounding boxes can be wrapped
    as a `MultiBx` using:
        `mbx(coords, label)`.
    :param coords: can be nested coordinates of type `list` of `list`s/`json` strings
        (`list`s of `dict`s)/`ndarray`s representing multiple boxes.
        If passing a list/json each index of the object should be of the following formats:
        - `list` can be formatted with `label`: `[x_min, y_min, x_max, y_max, label]`
            or without `label`: `[x_min, y_min, x_max, y_max]`
        - `dict` should be in `pascal_voc` format using the keys
            {"x_min": 0, "y_min": 0, "x_max": 1, "y_max": 1, "label": 'none'}
        If passing an `ndarray`, it should be of shape `(N,4)`.

    :param label: a `list` of `str`s that has the class name or label for the object in the
    corresponding box.
    """

    def __init__(self, coords, label: list = None, no_check=False):
        self.index = 0
        self._setup_complete = False
        self.no_check = no_check
        label = label if label else ["unknown"] * len(coords)  # default labels
        # TODO: need a good check for verifying multiple boxes are passed.
        # assert len(coords)>1 and len(label)>1, f"Expected multiple boxes in `MultiBx`: {coords}"
        assert isinstance(coords, ALL_ITER_TYPES)
        self.coords = coords
        self.label = label
        self.__setup__()

    def __setup__(self):
        """Setup the BaseBx for each item at index idx."""
        # anything here
        self._setup_complete = True

    def __len__(self):
        """Gets the length of coordinates."""
        return len(self.label)

    def __getitem__(self, idx):
        return coord_parser(self.coords[idx], self.label[idx], no_check=self.no_check)

    def __iter__(self):
        """Iterates through the boxes in `MultiBx` where self.valid is True."""
        return self

    def __next__(self):
        """Iteration is allowed only for valid boxes"""
        try:
            b = self[self.index]
            if not b.valid:
                # 0 area boxes are not valid
                self.index += 1
                return self.__next__()
        except IndexError:
            self.index = 0  # reset index
            raise StopIteration
        self.index += 1
        return b

    def __str__(self):
        return f"MultiBx(coords: {len(self.coords)}, labels: {len(self.label)})"

    def __repr__(self):
        return self.__str__()

    @property
    def shape(self):
        """Returns the shape of coordinates"""
        # TODO: check for number of points coordinates
        return len(self.coords), 4

In [ ]:
# | export
BX_TYPE = (Bx, MultiBx)

Generate random coordinates:

In [ ]:
np.random.seed(42)
annots = [sorted([np.random.randint(100) for i in range(4)]) for j in range(3)]
annots

[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]]

All annotations are stored as a `BaseBx` in a container called `MultiBx`

In [ ]:
bxs = MultiBx(annots, ["apple", "coke", "tree"])
bxs

MultiBx(coords: 3, labels: 3)

If no labels are passed, `unknown` is assigned. 

In [ ]:
bxs = MultiBx(annots)
bxs

MultiBx(coords: 3, labels: 3)

If a single bounding box is passed, `AssertionError` is NOT rasied anymore. 
The previous philosophy was that a single annotaiton `BaseBx` cannot be a `MultiBx`. 
It is upto the user to provide a list of list and not a list. There are better checks with `get_bx`.

In [ ]:
try:
    b = MultiBx([annots[0]])
    print(f"box {annots[0]} is {b}")
except AssertionError as A:
    print(A)

box [14, 51, 71, 92] is MultiBx(coords: 1, labels: 1)


Each index reveals the stored coordinate as a `BaseBx`

In [ ]:
bxs[0], bxs.shape

(BaseBx(coords=[[14, 51, 71, 92]], label=['unknown']), (3, 4))

In [ ]:
bxs.label

['unknown', 'unknown', 'unknown']

They can also be iterated:

In [ ]:
next(bxs)

BaseBx(coords=[[14, 51, 71, 92]], label=['unknown'])

Or using list comprehension, properties of individual boxes can be extracted

In [ ]:
[b.area for b in bxs]

[1612, 325]

In [ ]:
bxs[0].valid

True

In [ ]:
bxs[1].yolo()

[[51.0, 73.0, 62.0, 26.0, 'unknown']]

In [ ]:
bxs[0].area

2337

In [ ]:
annots_json = json.load(open("../data/annots.json"))
annots_json

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

If lists of lists are passed.

In [ ]:
list_bboxes = annots
list_bboxes

[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]]

In [ ]:
mbx_list = MultiBx(list_bboxes)
mbx_list

MultiBx(coords: 3, labels: 3)

In [ ]:
mbx_list[0]

BaseBx(coords=[[14, 51, 71, 92]], label=['unknown'])

If dicts are passed.

In [ ]:
dict_bboxes = (
    {"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"},
    {
        "x_min": 20,
        "y_min": 30,
        "x_max": 50,
        "y_max": 90,
        "label": "apple",
    },
)

In [ ]:
mbx_dict = MultiBx(dict_bboxes)
mbx_dict

MultiBx(coords: 2, labels: 2)

In [ ]:
mbx_dict[0]

BaseBx(coords=[[14, 51, 71, 92]], label=['item'])

If json strings are passed.

In [ ]:
json_bboxes = (
    '{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}',
    '{"x_min": 20, "y_min": 30, "x_max": 50, "y_max": 90, "label": "apple"}',
)

In [ ]:
mbx_json = MultiBx(json_bboxes)
mbx_json

MultiBx(coords: 2, labels: 2)

In [ ]:
mbx_json[0]

BaseBx(coords=[[14, 51, 71, 92]], label=['item'])

In [ ]:
json_bboxes_file = json.load(open("../data/annots.json"))
json_bboxes_file

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

In [ ]:
mbx_json_file = MultiBx(json_bboxes_file)
mbx_json_file

MultiBx(coords: 2, labels: 2)

In [ ]:
mbx_json_file[0], mbx_json_file[1]

(BaseBx(coords=[[130, 63, 225, 180]], label=['clock']),
 BaseBx(coords=[[13, 158, 90, 213]], label=['frame']))

Also accepts keys (for the dict) as a list, otherwise uses `voc_keys`.

In [ ]:
voc_keys

['x_min', 'y_min', 'x_max', 'y_max', 'label']

Defining shortcut function to process lists and dicts in `MultiBx`.

In [ ]:
# | export


def mbx(coords=None, label=None, no_check=False):
    """Alias of the `MultiBx` class."""
    return MultiBx(coords, label, no_check=no_check)

In [ ]:
annots_list = annots
annots_list

[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]]

In [ ]:
mbx_list = mbx(annots_list)

In [ ]:
mbx_list.coords

[[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99]]

When the annotation file contains the key `label`

In [ ]:
annots_json = json.load(open("../data/annots.json"))
annots_json

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

Still preserves the original format

In [ ]:
mbx_json_ = mbx(annots_json)

In [ ]:
mbx_json.coords

('{"x_min": 14, "y_min": 51, "x_max": 71, "y_max": 92, "label": "item"}',
 '{"x_min": 20, "y_min": 30, "x_max": 50, "y_max": 90, "label": "apple"}')

In [ ]:
# known issue
mbx_json.label

['unknown', 'unknown']

In [ ]:
mbx_json[1]  # after indexing the 1st index, the label gets assigned to multibx

BaseBx(coords=[[20, 30, 50, 90]], label=['apple'])

# `get_bx`

When in doubt, use `get_bx`.

In [ ]:
ITER_TYPES

(numpy.ndarray, list, fastcore.foundation.L)

In [ ]:
# | export


def get_bx(coords, label=None, no_check=False):
    """
    Helper function to check and call the correct type of Bx instance.

    Checks for the type of data passed and calls the respective class
    to generate a Bx instance. Currently only supports ndarray, list, dict,
    tuple, nested list, nested tuple.

    Parameters
    ----------
    coords : ndarray, list, dict, tuple, nested list, nested tuple
        Coordinates of anchor boxes.
    label : list, optional
        Labels for anchor boxes in order, by default None

    Returns
    -------
    Bx
        An instance of MultiBx, ListBx, BaseBx or JsonBx

    Raises
    ------
    NotImplementedError
        If unknown type of coordinates are passed.
    """
    # process ndarray
    if isinstance(coords, np.ndarray):
        coords = np.atleast_2d(coords)
        return mbx(coords, label, no_check)
    # process list
    if isinstance(coords, (list, L)):
        if isinstance(coords[0], COORD_TYPES):
            """If first item is a position"""
            return bbx(coords, label, no_check)
        elif isinstance(coords[0], ITER_TYPES + ITER_TYPES_EXTRA):
            """If fist item is an iterable"""
            return mbx(coords, label, no_check)
        elif isinstance(coords[0], ITER_TYPES_TUPLE):
            """If first item is a tuple"""
            return mbx([list(c) for c in coords], label, no_check)
    # process dict
    if isinstance(coords, dict):
        return bbx([coords], label, no_check)
    # process tuple
    if isinstance(coords, tuple):
        return bbx(list(coords), label, no_check)
    # process BX_TYPE
    if isinstance(coords, BX_TYPE):
        return coords
    else:
        raise NotImplementedError(
            f"{inspect.stack()[0][3]} of {__name__}: Got coords={coords} of type {type(coords)}."
        )

`get_bx` runs a bunch of if-else statements to call the right module when in doubt.

In [ ]:
annots_json

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

In [ ]:
get_bx(annots_json)

MultiBx(coords: 2, labels: 2)

In [ ]:
len(annots_json[0])

5

In [ ]:
get_bx([annots_json[0]])

MultiBx(coords: 1, labels: 1)

In [ ]:
get_bx(annots_list)

MultiBx(coords: 3, labels: 3)

In [ ]:
get_bx([0, 1, 1, 2, "juice"])  # boxes should be atleast of 1px length

BaseBx(coords=[[0, 1, 1, 2]], label=['juice'])

The below doesnt fail immediately but fails upon indexing into the box

In [ ]:
get_bx([[0, 1, 1, 1]])  # two brackets mean we have a multibx now

MultiBx(coords: 1, labels: 1)

Coming back to the original problem of the users burden in deciding which function to call:
```py
try:
    b = MultiBx([annots[0]])
    print(f"box {annots[0]} is {b}")
except AssertionError as A:
    print(A)
```

`get_bx` addresses this the following way:

In [ ]:
mbx(annots[0])  # this is wrong, as this is a single coordinate!

MultiBx(coords: 4, labels: 4)

In [ ]:
get_bx(annots[0])

BaseBx(coords=[[14, 51, 71, 92]], label=['unknown'])

In [ ]:
get_bx([annots[0]])

MultiBx(coords: 1, labels: 1)

The addition operation stacks the bounding boxes.

In [ ]:
# | export
@patch
def __add__(self: BaseBx, other):
    """Pseudo-add method that stacks the provided boxes and labels. Stacking two
    boxes imply that the resulting box is a `MultiBx`: `BaseBx` + `BaseBx`
    = `MultiBx`. This violates the idea of `BaseBx` since the result
    holds more than 1 coordinate/label for the box.
    From `v.2.0`, a `UserWarning` is issued if called.
    Recommended use is either: `BaseBx` + `BaseBx` = `MultiBx` or
    `basics.stack_bxs()`.
    """
    if not isinstance(other, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected a subclass of {BX_TYPE}"
        )
    else:
        warnings.warn(
            BxViolation(
                f"Change of object type imminent if trying to add "
                f"{type(self)}+{type(other)}. Use {type(other)}+{type(self)} "
                f"instead or basics.stack_bxs()."
            )
        )
    coords = self.coords + other.coords
    label = self.label + other.label
    return mbx(coords, label)


@patch
def __add__(self: MultiBx, other):
    """Pseudo-add method that stacks the provided boxes and labels. Stacking two
    boxes imply that the resulting box is a `MultiBx`: `MultiBx` + `MultiBx`
    = `MultiBx`. Same as `basics.stack_bxs()`.
    """
    if not isinstance(other, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, "
            f"got self={type(self)}, other={type(other)}"
        )
    coords = self.coords + other.coords
    label = self.label + other.label
    return mbx(coords, label)

In [ ]:
# | export
def stack_bxs(b1, b2):
    """
    Method to stack two Bx-types together. Similar to `__add__` of BxTypes
    but avoids UserWarning.
    :param b1:
    :param b2:
    :return:
    _summary_

    Parameters
    ----------
    b1 : Bx, MultiBx
        Anchor box coordinates Bx
    b2 : Bx, MultiBx
        Anchor box coordinates Bx

    Returns
    -------
    MultiBx
        Stacked anchor box coordinates of MultiBx type.

    Raises
    ------
    TypeError
        If unknown type of coordinates are passed.
    """

    if not isinstance(b1, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, got b1={type(b1)}"
        )
    if not isinstance(b2, BX_TYPE):
        raise TypeError(
            f"{inspect.stack()[0][3]} of {__name__}: Expected type {BX_TYPE}, got b2={type(b2)}"
        )
    if isinstance(b1, BaseBx):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            return b1 + b2
    return b1 + b2


def add_bxs(b1, b2):
    """Alias of stack_bxs()."""
    return stack_bxs(b1, b2)

In [ ]:
b

BaseBx(coords=[[10, 20, 100, 200]], label=['apple'])

Internally this is what is done to stack them:

In [ ]:
bxs.coords + b.coords, bxs.label + b.label

([[14, 51, 71, 92], [20, 60, 82, 86], [74, 74, 87, 99], [10, 20, 100, 200]],
 ['unknown', 'unknown', 'unknown', 'apple'])

In [ ]:
bxs + b

MultiBx(coords: 4, labels: 4)

Adding a `MultiBx` to a `BaseBx` makes the new set of coordinates a `MultiBx`, so a `BxViolation` warning is issued
if this was not intended. 

In [ ]:
b + bxs

/tmp/ipykernel_312689/55513013.py:17: BxViolation: Change of object type imminent if trying to add <class '__main__.BaseBx'>+<class '__main__.MultiBx'>. Use <class '__main__.MultiBx'>+<class '__main__.BaseBx'> instead or basics.stack_bxs().
  warnings.warn(


MultiBx(coords: 4, labels: 4)

In [ ]:
stack_bxs(b, bxs)

MultiBx(coords: 4, labels: 4)

To avoid the `BxViolation`, use the method `stack_bxs`.

In [ ]:
stack_bxs(bxs, b)

MultiBx(coords: 4, labels: 4)

In [ ]:
# | export
def stack_bxs_inplace(b, *args):
    """Stack the passed boxes on top of the first item."""
    for b_ in args:
        b = stack_bxs(b, b_)
    return b

In [ ]:
boxes = [
    BaseBx(coords=[200, 100, 300, 200], label=["a_3x3_1.0_5"]),
    BaseBx(coords=[214, 79, 285, 220], label=["a_3x3_0.5_5"]),
    BaseBx(coords=[222, 58, 277, 241], label=["a_3x3_0.3_5"]),
]

In [ ]:
stacked_bxs = stack_bxs_inplace(*boxes)

In [ ]:
stacked_bxs

MultiBx(coords: 3, labels: 3)

In [ ]:
stacked_bxs.coords

[[200, 100, 300, 200], [214, 79, 285, 220], [222, 58, 277, 241]]

In [ ]:
len(stacked_bxs)

3

`BaseBx` also supports calculation of bounding box offset by calling the `get_offset()` method.

In [ ]:
b1 = b
b2 = BaseBx(
    [18, 44, 71, 90], "probably_flower"
)  # simulating the case where we might have a bbox prediction
b1, b2

(BaseBx(coords=[[10, 20, 100, 200]], label=['apple']),
 BaseBx(coords=[[18, 44, 71, 90]], label=['probably_flower']))

In [ ]:
(b.coords_as_numpy - b2.coords_as_numpy)

array([[ -8, -24,  29, 110]])

In [ ]:
(b.coords_as_numpy - b2.coords_as_numpy) / np.tile([2, 1], 2)

array([[ -4. , -24. ,  14.5, 110. ]])

In [ ]:
# | export


@patch
def get_offset(
    self: BaseBx,
    other: BaseBx,
    normalize=True,
    log_func=np.log,
    sigma=(0.1, 0.2),
    self_is_anchor=False,
):
    """
    Caclulates the offset of the box I with another box O.
    The most basic calculation of offset involves a) taking the distance between the centers: `I_cx - O_cx`, `I_cy - O_cy`.
    b) taking the ratio of the two boxes: `I_w/Ow, I_h/O_h`.

    If `normalize=True`, the center distances and ratios are normalized as per https://arxiv.org/pdf/1512.02325.pdf
    `(I_cx - O_cx)/O_w`, `(I_cy - O_cy)/O_h`, `log(I_w/Ow), log(I_h/O_h)`
    These are further scaled with an appoximation of standard deviation for the distances and ratios
    `((I_cx - O_cx)/O_w)/sigma_c`, `((I_cy - O_cy)/O_h)/sigma_c`, `log(I_w/Ow)/sigma_r, log(I_h/O_h)/sigma_r`

    Args:
        other (BaseBx): Any supported type of bounding box format, even takes a list of coordinates. Typically the anchor box.
        normalize (bool, optional): Whether to normalize the offsets. Defaults to True.
        log_func (func, optional): Function for normalizing the ratio of widths and heights. Defaults to np.log.
        sigma (tuple, optional): Estimated of standard deviation for the distances and ratios. Defaults to (0.1, 0.2).
        self_is_anchor (bool, optional): Typically `other` is assumed to be the anchor box, this flag tells that this assumption is False. Defaults to False.

    Returns:
        list: Offsets of the two bounding boxes
    """
    if isinstance(other, MultiBx):
        warnings.warn(BxViolation(f"Other should be BaseBx, got MultiBx"))
        assert len(other) == 1, f"{other} cannot be converted to single bounding box."
        other = other[0]
    elif not isinstance(other, Bx):
        other = bbx(other)

    if self_is_anchor:
        # if self_is_anchor, ie anchor.get_offset(ground_truth) is called
        anchor = self
        gt = other
    else:
        # if not self_is_anchor, ie ground_truth.get_offset(anchor) is called (default behaviour)
        gt = self
        anchor = other
    # get anchor box w and h
    anchor_bw_norm = anchor.bw
    anchor_bh_norm = anchor.bh
    sigma_c, sigma_r = sigma
    # if not normalize, reset params
    if not normalize:
        log_func = noop
        sigma_c, sigma_r, anchor_bw_norm, anchor_bh_norm = [1.0] * 4
    # center distances
    # norm with anchor box w and h
    cx_offset, cy_offset = (
        (gt.cx - anchor.cx) / anchor_bw_norm,
        (gt.cy - anchor.cy) / anchor_bh_norm,
    )
    # scale of boxes
    w_offset = log_func(gt.bw / anchor_bw_norm)
    h_offset = log_func(gt.bh / anchor_bh_norm)

    offset = np.asarray([cx_offset, cy_offset, w_offset, h_offset])
    # norm with sigmaxy and sigmawh
    # print(sigma_c, sigma_r)
    offset /= np.repeat([sigma_c, sigma_r], 2)
    # not np.tile as norm is cx/sigma_c, cy/sigma_c, w/sigma_r, h/sigma_r
    return L(offset.round(4).tolist())

In [ ]:
np.repeat([1, 2], 2)

array([1, 1, 2, 2])

In [ ]:
isinstance(b, BaseBx)

True

In [ ]:
b, b2

(BaseBx(coords=[[10, 20, 100, 200]], label=['apple']),
 BaseBx(coords=[[18, 44, 71, 90]], label=['probably_flower']))

In [ ]:
b.get_offset(b2, normalize=False)

(#4) [10.5,43.0,90.0,180.0]

In [ ]:
b.get_offset(b2, normalize=True)

(#4) [1.9811,9.3478,2.6476,6.8216]

To make sure consistency with the reverse case where self is the ground truth anchor box and ground truth bounding box is the other box, use the `self_is_anchor` flag.

In [ ]:
b2.get_offset(b, normalize=True, self_is_anchor=True)  # correct

(#4) [1.9811,9.3478,2.6476,6.8216]

If `self_is_anchor=False` flag is not turned on, when ground truth bounding box is the other box, the latter can lead to a systematic bug.

In [ ]:
b2.get_offset(b, normalize=True)  # not correct

(#4) [-1.1667,-2.3889,-2.6476,-6.8216]

In [ ]:
# |hide
from nbdev import nbdev_export

nbdev_export()